<img src="https://raw.githubusercontent.com/seismo-live/seismo_live/refs/heads/master/notebooks/Workshops/2017_Baku_STCU_IRIS_ObsPy_course/images/obspy_logo_full_524x179px.png" width=90%> 

![](images/obspy_logo_full_524x179px.png)

In [1]:
try:
  import obspy
  import cartopy
except ModuleNotFoundError:
  !pip -qq install obspy cartopy
  import obspy
  import cartopy

In [2]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

#### FDSN Client Exercise

Use the FDSN client to assemble a waveform dataset for on event.

- search for a large earthquake (e.g. by depth or in a region of your choice, use option **`limit=5`** to keep network traffic down)

In [ ]:
from obspy.clients.fdsn import Client

client = Client()
catalog = client.get_events(minmagnitude=7, limit=5, mindepth=400)
print(catalog)
catalog.plot()
event = catalog[0]
print(event)

- search for stations to look at waveforms for the event. stations should..
    * be available at the time of the event
    * have a vertical 1 Hz stream ("LHZ", to not overpower our network..)
    * be in a narrow angular distance around the event (e.g. 90-91 degrees)
    * adjust your search so that only a small number of stations (e.g. 3-6) match your search criteria

In [ ]:
origin = event.origins[0]
t = origin.time

inventory = client.get_stations(longitude=origin.longitude, latitude=origin.latitude,
                                minradius=101, maxradius=102,
                                starttime=t, endtime =t+100,
                                channel="LHZ", matchtimeseries=True)
print(inventory)

- for each of these stations download data of the event, e.g. a couple of minutes before to half an hour after the event
- put all data together in one stream (put the `get_waveforms()` call in a try/except/pass block to silently skip stations that actually have no data available)
- print stream info, plot the raw data

In [ ]:
from obspy import Stream
st = Stream()

for network in inventory:
    for station in network:
        try:
            st += client.get_waveforms(network.code, station.code, "*", "LHZ",
                                       t - 5 * 60, t + 30 * 60, attach_response=True)
        except:
            pass

print(st)
st.plot();

- correct the instrument response for all stations and plot the corrected data

In [ ]:
st.remove_response(water_level=20)
st.plot()

If you have time, assemble and plot another similar dataset (e.g. like before stations at a certain distance from a big event, or use Transportable Array data for a big event, etc.)